## Initialization code
Run these three cells after you plug the ADALM into your computer

In [37]:
# import libraries needed for the code
import pysmu
from pysmu import Session, Mode, Device
import time
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import os
import math

In [38]:
# initialize settings on the ADALM1000
session = Session(queue_size = 100000)
sample_rate = 100000
session.configure(sample_rate = sample_rate)
output = session.devices[0].channels['A'] #selects channel A as the one applying the stimulation
dev = session.devices[0]

SessionError: failed scanning and/or adding all supported devices: Permission denied

## Assign parameters for each channel
Each section below is a different channel, add channels and edit parameters as needed
Code will print out the parameters for each channel

In [39]:
#Edit this code to change the channel parameters
num_of_channels = 2 
voltages = [3,3] #Unit: V
stim_durations = [0.01,0.01] #Unit: s
number_of_stims = [3600,7200] 
frequency = 1 #Unit: Hz

In [40]:
#Dont edit this code, it will calculate channel settings for you
#This code makes arbitrary waveforms for all channels
stim_waveforms = []
sample_number = []
for c in range(num_of_channels):
    waveform = [voltages[c]]*int((stim_durations[c]*100000))+[0]*500
    stim_waveforms.append(waveform)
    sample_number.append(len(waveform))

#assign arrays to each pin output
pin1 = [0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1]
pin2 = [0,0,1,1,0,0,1,1,0,0,1,1,0,0,1,1]
pin3 = [0,0,0,0,1,1,1,1,0,0,0,0,1,1,1,1]
pin4 = [0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1]

bool_lists = []
for c in range(num_of_channels):
    bool_lists.append([True]*number_of_stims[c]+[False]*(max(number_of_stims)-number_of_stims[c]))
bool_lists_by_period = [list(group) for group in zip(*bool_lists)]

## Test the connection of each device
Running the code below will apply 100mV for 1ms to each device and record the average current see if your devices are shorted or disconnected 

In [47]:
#assign arrays to each pin output
pin1 = [0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1]
pin2 = [0,0,1,1,0,0,1,1,0,0,1,1,0,0,1,1]
pin3 = [0,0,0,0,1,1,1,1,0,0,0,0,1,1,1,1]
pin4 = [0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1]

#Test for shorting
n = 100
for c in range(num_of_channels):
    # set output of the mux to the channel that should be stimmed
    D0 = 0x50 + pin1[c] # 0x50 = set to 0, 0x51 = set to 1
    D1 = 0x50 + pin2[c]
    D2 = 0x50 + pin3[c]
    D3 = 0x50 + pin4[c]
    dev.ctrl_transfer(0x40, D0, 4, 0, 0, 0, 100) # set PIO 0
    dev.ctrl_transfer(0x40, D1, 5, 0, 0, 0, 100) # set PIO 1
    dev.ctrl_transfer(0x40, D2, 6, 0, 0, 0, 100) # set PIO 2
    dev.ctrl_transfer(0x40, D3, 7, 0, 0, 0, 100) # set PIO 3 
    
    output.mode = Mode.SVMI
    output.constant(0.1)
    recording = output.get_samples(n)
    # print(recording)
    currents = []
    for number in range(n):
        currents.append(recording[number][1])
    avg_current = sum(currents)/len(currents)
    print(avg_current)
    if avg_current > 0.01:
        print(f"Channel {c+1} is shorted")
    elif avg_current < 0.0004:
        print(f"Channel {c+1} is disconnected")
    else:
        print(f"Channel {c+1} is connected")
    output.constant(0)
    output.get_samples(10)
    time.sleep(0.5)

0.0008676158879825379
Channel 1 is connected
0.0008584205017541536
Channel 2 is connected


## Stimulate the Devices
This is the actual stimulation code, don't edit and don't run until you're sure everything is set up!

In [48]:
#make new folder for stim data in the same location as this code
day = datetime.now().date()
newpath = f'{day}' 
if not os.path.exists(newpath):
    os.makedirs(newpath)


#start the stimulation
for s in range(max(number_of_stims)): #set total number of stimulations
    time_stimulating = 0
    bool_for_period = bool_lists_by_period[s]
    #cycle through channels
    for c in range(num_of_channels):
        #determine if the channel should be stimulated at this time  
        if bool_for_period[c]==True:
            start_time = time.time()
            
            # set output of the mux to the channel that should be stimmed
            D0 = 0x50 + pin1[c] # 0x50 = set to 0, 0x51 = set to 1
            D1 = 0x50 + pin2[c]
            D2 = 0x50 + pin3[c]
            D3 = 0x50 + pin4[c]
            dev.ctrl_transfer(0x40, D0, 4, 0, 0, 0, 100) # set PIO 0
            dev.ctrl_transfer(0x40, D1, 5, 0, 0, 0, 100) # set PIO 1
            dev.ctrl_transfer(0x40, D2, 6, 0, 0, 0, 100) # set PIO 2
            dev.ctrl_transfer(0x40, D3, 7, 0, 0, 0, 100) # set PIO 3 

            #stimulate channel with voltage pulses
            output.mode = Mode.SVMI
            output.arbitrary(stim_waveforms[c],cyclic=True)
            data = output.get_samples(sample_number[c])

            if s%(3600/(2*frequency))==0: #record every 30 mins
                print(f"Recording Ch{c+1}")
                file = "Recording_Ch{}".format(c+1)
                now = datetime.now()
                now_formatted = now.strftime("%Y_%m_%d__%H_%M_%S")
                V = []
                A = []
                for i in range(int(sample_number[c])):
                    V.append(data[i][0])
                    A.append(data[i][1])
                df = pd.DataFrame({"V" : V, "A": A})
                df.to_csv(f"{day}/ch{c+1}_{now_formatted}.csv")
            time_stimulating += time.time()-start_time
    time.sleep((1/frequency)-time_stimulating)

Recording Ch1
Recording Ch2


Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x110f1b820>>
Traceback (most recent call last):
  File "/opt/anaconda3/envs/libsmu_test/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 


KeyboardInterrupt: 